In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import Bio
import Bio.SeqIO
import re
import os

from timeit import default_timer as timer

/tmp/ipykernel_258205/3122067165.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader


/home/yisupeng/.conda/envs/py310/lib/python3.10/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/yisupeng/.conda/envs/py310/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/yisupeng/.conda/envs/py310/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMP

In [4]:
hyper_aa_regex = re.compile('[BXZJUO]')

In [5]:
MAX_LEN = 20

In [6]:
import torch
x = torch.tensor([1, 2, 3])
x.to('cuda')

tensor([1, 2, 3], device='cuda:0')

In [7]:
seqdb = {}
irregs = 0
for record in Bio.SeqIO.parse("data/uniprot_sprot.fasta", "fasta"):
    _, acc, geneid = record.id.split('|')
    if hyper_aa_regex.findall(str(record.seq)):
        irregs += 1
        continue
    if len(record.seq) > MAX_LEN:
        irregs += 1
        continue
    seqdb[acc] = record
print('irregs', irregs)

irregs 567803


In [8]:
len(seqdb)

3479

In [9]:
def build_datapoint(s):
    return ['>'] + list(s) + ['<']

In [10]:
def build_dataset(seqdb):
    res = []
    for acc, rec in seqdb.items():
        res.append(build_datapoint(str(rec.seq)))
    return res

In [11]:
for rec in seqdb.values():
    if 'B' in rec.seq:
        print(rec)
        break

In [12]:
ds = build_dataset(seqdb)
ds

[['>', 'G', 'L', 'E', 'E', 'T', 'Y', 'C', 'S', 'M', 'K', 'I', 'K', '<'],
 ['>',
  'G',
  'L',
  'E',
  'E',
  'T',
  'Y',
  'C',
  'S',
  'M',
  'R',
  'I',
  'K',
  'E',
  'N',
  'I',
  '<'],
 ['>', 'A', 'G', 'E', 'P', 'A', 'N', 'N', 'E', 'D', 'R', 'F', 'N', 'Y', '<'],
 ['>', 'L', 'D', 'V', 'E', 'L', 'T', 'V', 'E', 'E', 'R', '<'],
 ['>',
  'L',
  'A',
  'E',
  'Q',
  'A',
  'E',
  'R',
  'Y',
  'E',
  'E',
  'M',
  'V',
  'E',
  'F',
  'M',
  'E',
  'K',
  '<'],
 ['>',
  'A',
  'L',
  'A',
  'G',
  'T',
  'I',
  'I',
  'A',
  'G',
  'A',
  'S',
  'L',
  'T',
  'F',
  'K',
  'I',
  'L',
  'D',
  'E',
  'V',
  '<'],
 ['>',
  'M',
  'K',
  'L',
  'I',
  'A',
  'K',
  'T',
  'R',
  'I',
  'L',
  'N',
  'T',
  'L',
  'V',
  'F',
  'S',
  'A',
  'A',
  'G',
  'K',
  '<'],
 ['>',
  'P',
  'A',
  'Q',
  'P',
  'F',
  'R',
  'I',
  'K',
  'K',
  'R',
  'Q',
  'G',
  'P',
  'F',
  'E',
  'R',
  'P',
  '<'],
 ['>',
  'P',
  'A',
  'Q',
  'P',
  'F',
  'R',
  'F',
  'P',
  'K',
  'H',
  'P',
  'Q

In [13]:
vocab = set(c for s in ds for c in s)

In [14]:
vocab

{'<',
 '>',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y'}

In [15]:
len(vocab)

22

In [16]:
from toy_transformer import *

2.3.1+cu121


In [17]:
src_vocab_size = len(vocab)
target_vocab_size = len(vocab)
num_layers = 6
seq_length = 12

In [18]:
torch.triu(torch.ones((5, 5), device=DEVICE))

tensor([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]], device='cuda:0')

In [19]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask
def generate_square_diagnal_mask(sz):
    mask = (torch.diag(torch.ones(sz, device=DEVICE)))
    mask = mask.float().masked_fill(mask == 1, float('-inf')).masked_fill(mask == 0, float(0.0))
    return mask

In [20]:
generate_square_subsequent_mask(5)

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]], device='cuda:0')

In [21]:
generate_square_diagnal_mask(5)

tensor([[-inf, 0., 0., 0., 0.],
        [0., -inf, 0., 0., 0.],
        [0., 0., -inf, 0., 0.],
        [0., 0., 0., -inf, 0.],
        [0., 0., 0., 0., -inf]], device='cuda:0')

In [22]:
def create_one_out_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


In [23]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List

In [24]:
def yield_tokens(data_iter: Iterable) -> List[str]:
    for data_sample in data_iter:
        yield list(data_sample)
    

In [25]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['X', '_', '>', '<']

In [26]:
train_iter = seqdb.values()
vocab_transform = build_vocab_from_iterator(yield_tokens(train_iter),
                                            min_freq=1,
                                            specials=special_symbols,
                                            special_first=True)

In [27]:
vocab_transform.set_default_index(UNK_IDX)

In [28]:
token_transform = list

In [29]:
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input

    return func


In [30]:
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))
def pad_transform(tokens):
    return tokens + [PAD_IDX] * (seq_length - len(tokens))

text_transform = sequential_transforms(token_transform,  # Tokenization
                                       vocab_transform,  # Numericalization
                                       # pad_transform,
                                       tensor_transform)

In [31]:
model = Transformer(embed_dim=512, src_vocab_size=src_vocab_size,
                    target_vocab_size=target_vocab_size, seq_length=seq_length,
                    num_layers=num_layers, expansion_factor=4, n_heads=8)


In [32]:
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for sample in batch:
        src_batch.append(text_transform(sample))
        tgt_batch.append(text_transform(sample))

    # pad_first_to_len(src_batch, seq_length)
    # pad_first_to_len(tgt_batch, seq_length)

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch



In [33]:
transformer = model

In [34]:
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)


In [35]:
transformer = transformer.to(DEVICE)


In [36]:
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)


In [37]:
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)


In [38]:
token_transform(next(iter(seqdb.values())))

['G', 'L', 'E', 'E', 'T', 'Y', 'C', 'S', 'M', 'K', 'I', 'K']

In [39]:
def generate_tokens(logits):
    tokens = torch.max(logits, dim=-1)[1]
    return tokens


def decode_tokens(tokens):
    words = vocab_transform.lookup_tokens(tokens.flatten().tolist())
    words = np.array(words).reshape(tokens.shape)
    return words



In [40]:
dataset = list(seqdb.values())

In [41]:
def evaluate(model):
    model.eval()
    losses = 0

    # val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_iter = ToyData(split='valid')
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.T.to(DEVICE)
        tgt = tgt.T.to(DEVICE)

        tgt_input = tgt[:, :-1]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_one_out_mask(src, tgt_input)

        # logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        logits = model(src, tgt_input)

        tgt_out = tgt[:, 1:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        # loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))


In [42]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    # train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_iter = dataset
    train_dataloader = DataLoader(train_iter, batch_size=20, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        print(src.shape, tgt.shape)
        src = src.T.to(DEVICE)
        tgt = tgt.T.to(DEVICE)

        # tgt_input = tgt[:, :-1]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_one_out_mask(src, tgt)

        # logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        logits = model(src, tgt)
        # logits = model(src, tgt[:, :])
        tokens = generate_tokens(logits)

        optimizer.zero_grad()

        tgt_out = tgt[:, 1:]
        print(decode_tokens(tgt_out))
        print(decode_tokens(tokens))
        loss = loss_fn(logits.transpose(-1, -2), tgt_out)
        # loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))



In [43]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [44]:
continue_previous = True
# continue_previous = False
if continue_previous and os.path.exists('transformer.pickle'):
    transformer = pickle.load(open('transformer.pickle', 'rb'))


from timeit import default_timer as timer

NUM_EPOCHS = 180

for epoch in range(1, NUM_EPOCHS + 1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print(( f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

pickle.dump(transformer, open('transformer.pickle', 'wb'))


torch.Size([22, 20]) torch.Size([22, 20])


RuntimeError: The size of tensor a (22) must match the size of tensor b (12) at non-singleton dimension 1